In [13]:
!conda list

# packages in environment at /root/miniconda3/envs/openmmlab:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
asttokens                 2.0.5              pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2022.07.19           h06a4308_0  
certifi                   2022.9.14        py38h06a4308_0  
cudatoolkit               11.1.74              h6bb024c_0    nvidia
debugpy                   1.5.1            py38h295c915_0  
decorator                 5.1.1              pyhd3eb1b0_0  
entrypoints               0.4              py38h06a4308_0  
executing                 0.8.3              pyhd3eb1b0_0  
ffmpeg                    4.2.2                h20bf706_0  
freetype             

### Understand behavior of Batch Normalization of pytorch 1.8.2, below is the document from pytorch

https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html?highlight=batchnorm2d#torch.nn.BatchNorm2d

Backend C++ source file:
https://github.com/pytorch/pytorch/blob/420b37f3c67950ed93cd8aa7a12e673fcfc5567b/aten/src/ATen/native/Normalization.cpp#L61-L126 

torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)[SOURCE]

Applies Batch Normalization over a 4D input (a mini-batch of 2D inputs with additional channel dimension) as described in the paper Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift .

y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta

The mean and standard-deviation are calculated per-dimension over the mini-batches and \gammaγ and \betaβ are learnable parameter vectors of size C (where C is the input size). By default, the elements of \gammaγ are set to 1 and the elements of \betaβ are set to 0. The standard-deviation is calculated via the biased estimator, equivalent to torch.var(input, unbiased=False).

Also by default, during training this layer keeps running estimates of its computed mean and variance, which are then used for normalization during evaluation. The running estimates are kept with a default momentum of 0.1.

If track_running_stats is set to False, this layer then does not keep running estimates, and batch statistics are instead used during evaluation time as well.

NOTE

This momentum argument is different from one used in optimizer classes and the conventional notion of momentum. Mathematically, the update rule for running statistics here is \hat{x}_\text{new} = (1 - \text{momentum}) \times \hat{x} + \text{momentum} \times x_t 
​
is the new observed value.

Because the Batch Normalization is done over the C dimension, computing statistics on (N, H, W) slices, it’s common terminology to call this Spatial Batch Normalization.


In [14]:
import torch
import torch.nn as nn
m = nn.BatchNorm2d(2, affine = False, momentum = 0.1, track_running_stats = True)
input1 = torch.tensor(
    [[[[1., 1.],
       [1., 1.]],
      [[1., 1.],
       [1., 1.]]], 
     [[[2., 2.], 
       [2., 2.]], 
      [[3., 3.], 
       [3., 3.]]]])
for i in range(100):
    # print(m.running_mean, m.running_var, m.momentum)
    o = m(input1)
print("after many batches and before call eval(), running_mean and running_var :", m.running_mean, m.running_var, m.momentum)
m.eval()
print("after many batches and after call eval(), running_mean and running_var :", m.running_mean, m.running_var, m.momentum)

# output: 
# after many batches and before call eval(), running_mean and running_var : tensor([1.5000, 1.9999]) tensor([0.2857, 1.1429]) 0.1
# after many batches and after call eval(), running_mean and running_var : tensor([1.5000, 1.9999]) tensor([0.2857, 1.1429]) 0.1

input1 = torch.tensor(
    [[[[1., 1.],
       [1., 1.]],
      [[2., 2.],
       [2., 2.]]]])

torch.set_printoptions(precision=4, sci_mode=False)
print("BatchNorm2d eval input1: ", m(input1))

# output, note: (1.0 - 1.5) / ((0.2857) ** 0.5) == -0.9353
# tensor([[[[    -0.9353,     -0.9353],
#           [    -0.9353,     -0.9353]],

#          [[     0.0000,      0.0000],
#           [     0.0000,      0.0000]]]])



after many batches and before call eval(), running_mean and running_var : tensor([1.5000, 1.9999]) tensor([0.2857, 1.1429]) 0.1
after many batches and after call eval(), running_mean and running_var : tensor([1.5000, 1.9999]) tensor([0.2857, 1.1429]) 0.1
BatchNorm2d eval input1:  tensor([[[[    -0.9353,     -0.9353],
          [    -0.9353,     -0.9353]],

         [[     0.0000,      0.0000],
          [     0.0000,      0.0000]]]])


In [15]:
import torch
import torch.nn as nn

m = nn.BatchNorm2d(2, affine = False, momentum = 0.1, track_running_stats = True)
# input1.shape (2, 2, 2, 2)
input1 = torch.tensor(
    [[[[1., 1.],
       [1., 1.]],
      [[1., 1.],
       [1., 1.]]], 
     [[[2., 2.], 
       [2., 2.]], 
      [[3., 3.], 
       [3., 3.]]]])
print(input1.shape)

for i in range(50):
    print(m.running_mean, m.running_var, m.momentum)
    m(input1)



torch.Size([2, 2, 2, 2])
tensor([0., 0.]) tensor([1., 1.]) 0.1
tensor([0.1500, 0.2000]) tensor([0.9286, 1.0143]) 0.1
tensor([0.2850, 0.3800]) tensor([0.8643, 1.0271]) 0.1
tensor([0.4065, 0.5420]) tensor([0.8064, 1.0387]) 0.1
tensor([0.5159, 0.6878]) tensor([0.7544, 1.0491]) 0.1
tensor([0.6143, 0.8190]) tensor([0.7075, 1.0585]) 0.1
tensor([0.7028, 0.9371]) tensor([0.6653, 1.0669]) 0.1
tensor([0.7826, 1.0434]) tensor([0.6274, 1.0745]) 0.1
tensor([0.8543, 1.1391]) tensor([0.5932, 1.0814]) 0.1
tensor([0.9189, 1.2252]) tensor([0.5624, 1.0875]) 0.1
tensor([0.9770, 1.3026]) tensor([0.5348, 1.0930]) 0.1
tensor([1.0293, 1.3724]) tensor([0.5099, 1.0980]) 0.1
tensor([1.0764, 1.4351]) tensor([0.4874, 1.1025]) 0.1
tensor([1.1187, 1.4916]) tensor([0.4673, 1.1065]) 0.1
tensor([1.1568, 1.5425]) tensor([0.4491, 1.1102]) 0.1
tensor([1.1912, 1.5882]) tensor([0.4328, 1.1134]) 0.1
tensor([1.2220, 1.6294]) tensor([0.4181, 1.1164]) 0.1
tensor([1.2498, 1.6665]) tensor([0.4048, 1.1190]) 0.1
tensor([1.2749, 1.6

In [16]:
import torch
import torch.nn as nn
m = nn.BatchNorm2d(2, affine = False, momentum = 1, track_running_stats = True)
# input1.shape (2, 2, 2, 2)
input1 = torch.tensor(
    [[[[1., 1.],
       [1., 1.]],
      [[1., 1.],
       [1., 1.]]], 
     [[[2., 2.], 
       [2., 2.]], 
      [[3., 3.], 
       [3., 3.]]]])
print(input1.shape)

for i in range(100):
    print(m.running_mean, m.running_var, m.momentum)
    m(input1)



torch.Size([2, 2, 2, 2])
tensor([0., 0.]) tensor([1., 1.]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
tensor([1.5000, 2.0000]) tensor([0.2857, 1.1429]) 1
ten

In [17]:
import torch
import torch.nn as nn
m = nn.BatchNorm2d(2, affine = False, momentum = 1, track_running_stats = False)
m.running_mean = nn.Parameter(torch.tensor([0.0, 0.0], requires_grad = False))
m.running_var = nn.Parameter(torch.tensor([0.5, 0.5], requires_grad = False))
# input1.shape (2, 2, 2, 2)
input1 = torch.tensor(
    [[[[1., 1.],
       [1., 1.]],
      [[1., 1.],
       [1., 1.]]], 
     [[[2., 2.], 
       [2., 2.]], 
      [[3., 3.], 
       [3., 3.]]]])
print(input1.shape)

for i in range(10):
    # print(m.running_mean, m.running_var, m.momentum)
    print(m(input1))

print(m(input1))



torch.Size([2, 2, 2, 2])
tensor([[[[-1.0000, -1.0000],
          [-1.0000, -1.0000]],

         [[-1.0000, -1.0000],
          [-1.0000, -1.0000]]],


        [[[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]],

         [[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]]]])
tensor([[[[-1.0000, -1.0000],
          [-1.0000, -1.0000]],

         [[-1.0000, -1.0000],
          [-1.0000, -1.0000]]],


        [[[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]],

         [[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]]]])
tensor([[[[-1.0000, -1.0000],
          [-1.0000, -1.0000]],

         [[-1.0000, -1.0000],
          [-1.0000, -1.0000]]],


        [[[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]],

         [[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]]]])
tensor([[[[-1.0000, -1.0000],
          [-1.0000, -1.0000]],

         [[-1.0000, -1.0000],
          [-1.0000, -1.0000]]],


        [[[ 1.0000,  1.0000],
          [ 1.0000,  1.0000]],

         [[ 1.0000,  1.0000],
    